
<img src="../../img/ods_stickers.jpg">

## <center> [mlcourse.ai](https://mlcourse.ai) – открытый курс OpenDataScience по машинному обучению 
    
Автор материала: Юрий Кашницкий (@yorko в Slack ODS). Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание 6 (демо). Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина</center>
<img src='../../img/wine_quality.jpg' width=30%>

**Заполните пропущенный код и ответьте на вопросы в [онлайн-форме](https://docs.google.com/forms/d/1gsNxgkd0VqidZp4lh9mnCQnJw3b0IFR1C4WBES86J40).**

In [150]:
# отключим всякие предупреждения Anaconda
import warnings

warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [151]:
data = pd.read_csv('./../data/winequality-white.csv', sep=';')

In [182]:
data.head()

# фикс. кисл / летучая кисл. / лимонная числота / остаточный сахар / хлориды / свободный диоксид серы / всего диоксда скры / 
# плотность / сульфаты / алкоголь / качество

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [153]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [154]:
independent_columns_names = data.columns.values
independent_columns_names = [x for x in data if x != "quality"]

**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [155]:
y = data['quality']

X = data.drop('quality', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [156]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train);

In [157]:
linreg_pred_train = linreg.predict(X_train_scaled)
linreg_pred_test = linreg.predict(X_test_scaled)

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [158]:
print("Ответ:")
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg_pred_train))
print("Mean squared error (test): %.3f" % mean_squared_error(y_test, linreg_pred_test))

Ответ:
Mean squared error (train): 0.558
Mean squared error (test): 0.584


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [160]:
linreg_coef = pd.DataFrame({
    'features': independent_columns_names,
    'coefs': list(map(abs, linreg.coef_.flatten().tolist()))
})

linreg_coef.sort_values(by='coefs', ascending=False)

,features,coefs
7,density,0.665720
3,residual sugar,0.538164
1,volatile acidity,0.192260
8,pH,0.150036
10,alcohol,0.129533
0,fixed acidity,0.097822
9,sulphates,0.062053
5,free sulfur dioxide,0.042180
6,total sulfur dioxide,0.014304
4,chlorides,0.008127


**Ответ:** density(0.665720)

## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [161]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train);

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [167]:
lasso1_coef = pd.DataFrame({
    'features': independent_columns_names,
    'coefs': list(map(abs, lasso1.coef_.flatten().tolist()))
})

lasso1_coef.sort_values(by='coefs', ascending=False)

,features,coefs
10,alcohol,0.322425
3,residual sugar,0.256363
7,density,0.235492
1,volatile acidity,0.188479
8,pH,0.067277
5,free sulfur dioxide,0.043088
9,sulphates,0.029722
4,chlorides,0.002747
0,fixed acidity,0.000000
2,citric acid,0.000000


**Ответ:** fixed acidity (отвалился первый, т.к. имеет наименьшее значение индекса)

**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [168]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, n_jobs=-1, random_state=17)
lasso_cv.fit(X_train_scaled, y_train);

In [169]:
print("Лучшее значение 𝛼: %f" % lasso_cv.alpha_)

Лучшее значение 𝛼: 0.000283


**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина.**<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [170]:
lasso_cv_coef = pd.DataFrame({
    'features': independent_columns_names,
    'coefs': list(map(abs, lasso_cv.coef_.flatten().tolist()))
})

lasso_cv_coef.sort_values(by='coefs', ascending=False)

,features,coefs
7,density,0.648161
3,residual sugar,0.526883
1,volatile acidity,0.192049
8,pH,0.146549
10,alcohol,0.137115
0,fixed acidity,0.093295
9,sulphates,0.060939
5,free sulfur dioxide,0.042698
6,total sulfur dioxide,0.012969
4,chlorides,0.006933


**Ответ:** citric acid

**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [171]:
y_lcv_train = lasso_cv.predict(X_train_scaled)
y_lcv_test = lasso_cv.predict(X_test_scaled)

In [172]:
print("Ответ:")
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, y_lcv_train))
print("Mean squared error (test): %.3f" % mean_squared_error(y_test, y_lcv_test))

Ответ:
Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.**

In [192]:
# т.к. это случайный лес, то использование нормализированных данных необязательно

forest = RandomForestRegressor(random_state=17, n_jobs=-1)
forest.fit(X_train, y_train);

**<font color='red'>Вопрос 5:</font> Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [193]:
y_rfc_train = forest.predict(X_train)
y_rfc_test = forest.predict(X_test)
y_rfc_cv = np.mean(np.abs(cross_val_score(forest, X_train, y_train, scoring='neg_mean_squared_error')))

In [194]:
print("Ответ:")
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, y_rfc_train))
print("Mean squared error (cv): %.3f" % y_rfc_cv)
print("Mean squared error (test): %.3f" % mean_squared_error(y_test, y_rfc_test))

Ответ:
Mean squared error (train): 0.053
Mean squared error (cv): 0.414
Mean squared error (test): 0.371


**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.**

In [198]:
forest = RandomForestRegressor(random_state=17)

forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(forest, param_grid=forest_params, n_jobs=1, verbose=2)
locally_best_forest.fit(X_train, y_train)

In [197]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6, 'min_samples_leaf': 1},
 0.4938417994321389)

**К сожалению, результаты  GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании *random_state*). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).**<br>
**<font color='red'>Вопрос 6:</font> Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?**

In [176]:
forest_best = RandomForestRegressor(max_depth=19, max_features=7, min_samples_leaf=1, random_state=17)
forest_best.fit(X_train, y_train);

In [177]:
y_rfc_train = forest_best.predict(X_train)
y_rfc_test = forest_best.predict(X_test)
y_rfgs = np.mean(np.abs(cross_val_score(forest_best, X_train, y_train, scoring='neg_mean_squared_error')))

In [178]:
print("Ответ:")
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, y_rfc_train))
print("Mean squared error (cv): %.3f" % y_rfgs)
print("Mean squared error (test): %.3f" % mean_squared_error(y_test, y_rfc_test))

Ответ:
Mean squared error (train): 0.058
Mean squared error (cv): 0.404
Mean squared error (test): 0.369


**Оцените важность признаков с помощью случайного леса.**<br>
**<font color='red'>Вопрос 7:</font> Какой признак оказался главным в настроенной модели случайного леса?**

In [180]:
rf_importance = pd.DataFrame({
    'importance': independent_columns_names,
    'coefs': list(map(float, forest_best.feature_importances_.flatten().tolist()))
})

rf_importance.sort_values(by='coefs', ascending=False)

,importance,coefs
10,alcohol,0.214825
1,volatile acidity,0.117627
5,free sulfur dioxide,0.113509
7,density,0.086530
8,pH,0.072646
3,residual sugar,0.072529
6,total sulfur dioxide,0.071644
4,chlorides,0.067819
0,fixed acidity,0.061873
2,citric acid,0.061500


**Ответ:** alcohol (0.214825)

**Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.**

**Вывод:** случайный лес справляется с задачей лучше, чем линейные модели, видимо, потому, что зависимость качества вина от признаков нелинейна